In [ ]:
# import nibabel as nib
# from google.colab import files

# # Upload a .nii.gz file
# uploaded = files.upload()

# # Extract the file name from the uploaded dictionary
# file_name = list(uploaded.keys())[0]

# # Load the NIfTI file using nibabel
# nii_img = nib.load(file_name)

# # Get the image data as a NumPy array
# img_data = nii_img.get_fdata()

# # Print the shape (Z, Y, X)
# print("Volume Shape:", img_data.shape)

In [ ]:
# import nibabel as nib
# from google.colab import files

# # Upload a .nii.gz file
# uploaded = files.upload()

# # Extract the file name from the uploaded dictionary
# file_name = list(uploaded.keys())[0]

# # Load the NIfTI file using nibabel
# nii_img = nib.load(file_name)

# # Get the image data as a NumPy array
# img_data = nii_img.get_fdata()

# # Print the shape (Z, Y, X)
# print("Volume Shape:", img_data.shape)

In [ ]:
# import tensorflow as tf

# device_name = tf.config.list_physical_devices('GPU')

# if not device_name:
#     raise SystemError('GPU device not found')

# print(f'Found GPU at: {device_name}')

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

In [ ]:
# # Uninstall old or conflicting versions
# %pip uninstall numpy tensorflow keras scikit-image patchify matplotlib scikit-learn nibabel segmentation-models-3D classification-models-3D efficientnet-3D -y

In [ ]:
# # Install fixed versions only
# %pip install numpy==1.24.4 
# %pip install tensorflow==2.13.0 
# %pip install keras==2.13.1 
# %pip install scikit-image==0.21.0
# %pip install patchify==0.2.3 
# %pip install matplotlib==3.7.1 
# %pip install scikit-learn==1.3.0 
# %pip install nibabel==5.1.0 
# %pip install segmentation-models-3D==1.0.4 
# %pip install classification-models-3D==1.0.2 
# %pip install efficientnet-3D==1.0.0


In [2]:
import numpy
import tensorflow
import keras
import skimage
import patchify
import matplotlib
import sklearn
import nibabel
import segmentation_models_3D
import classification_models_3D
import efficientnet_3D

print("numpy:", numpy.__version__)
print("tensorflow:", tensorflow.__version__)
print("keras:", keras.__version__)
print("scikit-image:", skimage.__version__)
print("matplotlib:", matplotlib.__version__)
print("scikit-learn:", sklearn.__version__)
print("nibabel:", nibabel.__version__)
print("segmentation-models-3D:", segmentation_models_3D.__version__)
print("classification-models-3D:", classification_models_3D.__version__)
print("efficientnet-3D:", efficientnet_3D.__version__)


Segmentation Models: using `tf.keras` framework.
numpy: 1.24.3
tensorflow: 2.13.0
keras: 2.13.1
scikit-image: 0.21.0
matplotlib: 3.7.1
scikit-learn: 1.3.0
nibabel: 5.1.0
segmentation-models-3D: 1.0.4
classification-models-3D: 1.0.0
efficientnet-3D: 1.0.0


In [ ]:
# %pip install classification-models-3D
# %pip install efficientnet-3D
# %pip install segmentation-models-3D
# %pip install patchify

In [ ]:
import segmentation_models_3D as sm

In [ ]:
from skimage import io
from patchify import patchify, unpatchify
import numpy as np
from matplotlib import pyplot as plt
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import nibabel as nib

In [ ]:
import nibabel as nib
import numpy as np
from patchify import patchify
import math

# Load the .nii.gz image and mask
image_nifti = nib.load('./Data_Subset/images/ct_1003_image.nii.gz')
mask_nifti = nib.load('./Data_Subset/labels/ct_1003_label.nii.gz')

# Convert to NumPy arrays
image = np.array(image_nifti.get_fdata(), dtype=np.float32)
mask = np.array(mask_nifti.get_fdata(), dtype=np.uint8)  # Assuming binary mask

# Get original dimensions
H, W, D = image.shape  # Shape = (Height, Width, Depth)
print(f"Original Image Shape: {image.shape}")

# Compute new depth (nearest multiple of 64)
new_depth = math.ceil(D / 64) * 64
depth_padding = new_depth - D

# Pad depth dimension with zeros (only in the last axis)
image_padded = np.pad(image, ((0, 0), (0, 0), (0, depth_padding)), mode='constant', constant_values=0)
mask_padded = np.pad(mask, ((0, 0), (0, 0), (0, depth_padding)), mode='constant', constant_values=0)

print(f"Padded Image Shape: {image_padded.shape}")

# Extract 3D patches of (64, 64, 64) with step size 64
image_patches = patchify(image_padded, (64, 64, 64), step=64)
mask_patches = patchify(mask_padded, (64, 64, 64), step=64)

print(f"Image patches shape before reshaping: {image_patches.shape}")
print(f"Mask patches shape before reshaping: {mask_patches.shape}")

# Reshape patches for training
image_patches = image_patches.reshape(-1, 64, 64, 64)  # Convert to (N, 64, 64, 64)
mask_patches = mask_patches.reshape(-1, 64, 64, 64)

# Add channel dimension (1 for grayscale images)
image_patches = image_patches[:, np.newaxis, :, :, :]  # Shape: (N, 1, 64, 64, 64)
mask_patches = mask_patches[:, np.newaxis, :, :, :]

print(f"Final Image Patches Shape: {image_patches.shape}")  # (N, 1, 64, 64, 64)
print(f"Final Mask Patches Shape: {mask_patches.shape}")  # (N, 1, 64, 64, 64)

# Total patches count
print(f"Total Image Patches: {image_patches.shape[0]}")
print(f"Total Mask Patches: {mask_patches.shape[0]}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Select a random patch index
patch_index = 114
sample_patch = image_patches[patch_index, 0]  # Shape: (64, 64, 64)
sample_mask = mask_patches[patch_index, 0]    # Corresponding mask patch

# Randomly select 3 depth slices
depth_slices = np.random.choice(range(64), 3, replace=False)

# Create a 2-row grid (3 columns): Top row (image slices), Bottom row (mask slices)
fig, axes = plt.subplots(2, 3, figsize=(12, 6))

for i in range(3):
    depth = depth_slices[i]

    # Image slice (Top row)
    axes[0, i].imshow(sample_patch[:, :, depth], cmap='plasma')
    axes[0, i].set_title(f"Image - Depth {depth}")
    axes[0, i].axis("off")

    # Mask slice (Bottom row)
    axes[1, i].imshow(sample_mask[:, :, depth], cmap='gray')
    axes[1, i].set_title(f"Mask - Depth {depth}")
    axes[1, i].axis("off")

plt.suptitle(f"Patch {patch_index}: Image & Mask Slices", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
print(image_patches.shape)
# [patches, channels, height , width, depth]
# 256 patches and 1 grayscale channel , 64 x 64 x 64 - as patch volumen size

In [ ]:
# import numpy as np
# import nibabel as nib

# # Load the label NIfTI file
# mask_nifti = nib.load('/content/drive/MyDrive/Image_CHD/Data_Subset/labels/ct_1001_label.nii.gz')
# label_data = mask_nifti.get_fdata()

# # Identify unique class labels
# unique_classes = np.unique(label_data)
# print(f"Segmentation classes: {unique_classes}")


In [ ]:
n_classes = 8

In [ ]:
# Convert grayscale images (1 channel) to 3 channels
train_img = np.repeat(image_patches, 3, axis=1)  # Shape: (256, 3, 64, 64, 64)

# One-hot encode masks
train_mask_cat = to_categorical(mask_patches, num_classes=n_classes)  # Shape: (256, 1, 64, 64, 64, 8)

# Remove redundant channel dimension for masks
train_mask_cat = train_mask_cat.squeeze(1)  # New shape: (256, 64, 64, 64, 8)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    train_img, train_mask_cat, test_size=0.10, random_state=0
)

X_train = np.transpose(X_train, (0, 2, 3, 4, 1))  # (230, 64, 64, 64, 3)
X_test = np.transpose(X_test, (0, 2, 3, 4, 1))    # (26, 64, 64, 64, 3)


print("X_train shape:", X_train.shape)  # ✅ (230, 64, 64, 64, 3)
print("y_train shape:", y_train.shape)  # ✅ (230, 64, 64, 64, 8)
print("X_test shape:", X_test.shape)    # ✅ (26, 64, 64, 64, 3)
print("y_test shape:", y_test.shape)    # ✅ (26, 64, 64, 64, 8)


In [ ]:
#Define parameters for our model.

encoder_weights = 'imagenet'
BACKBONE = 'vgg16'  
activation = 'softmax'
patch_size = 64
n_classes = 8
channels = 3

LR = 0.0001
optim = keras.optimizers.Adam(learning_rate = LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# Losses
dice_loss = sm.losses.DiceLoss(class_weights=np.array([1.0]*n_classes))
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + focal_loss

# Metrics
metrics = [
    sm.metrics.IOUScore(threshold=0.5),
    sm.metrics.FScore(threshold=0.5)
]

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss



In [ ]:
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:

#Preprocess input data - otherwise you end up with garbage resutls
# and potentially model that does not converge.
X_train_prep = preprocess_input(X_train)
X_test_prep = preprocess_input(X_test)

In [ ]:
#Define the model. Here we use Unet but we can also use other model architectures from the library.
model = sm.Unet(BACKBONE, classes = n_classes,
                input_shape = (patch_size, patch_size, patch_size, channels),
                encoder_weights = encoder_weights,
                activation = activation)

model.compile(optimizer = optim, loss=total_loss, metrics=metrics)
print(model.summary())

In [ ]:
print(X_test_prep.shape)
print(X_train_prep.shape)

In [ ]:
#Fit the model
history=model.fit(X_train_prep,
          y_train,
          batch_size = 8,
          epochs = 2,
          verbose = 1,
          validation_data = (X_test_prep, y_test))

In [ ]:
#Save model for future use
model.save('./3D_model_vgg16_2epochs.h5')


In [ ]:
###
#plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['iou_score']
val_acc = history.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

For loading the model and continue training

In [ ]:
# from keras.models import load_model
# import segmentation_models_3D as sm
# from keras.optimizers import Adam
# import numpy as np

# # Step 1: Load the model
# model = load_model('./3D_model_vgg16_2epochs.h5', compile=False)

# # Step 2: Re-define loss and metrics
# n_classes = 8
# dice_loss = sm.losses.DiceLoss(class_weights=np.array([1/n_classes] * n_classes))  # or your custom weights
# focal_loss = sm.losses.CategoricalFocalLoss()
# total_loss = dice_loss + focal_loss

# metrics = [
#     sm.metrics.IOUScore(threshold=0.5),
#     sm.metrics.FScore(threshold=0.5)
# ]

# # Step 3: Recompile the model
# model.compile(
#     optimizer=Adam(learning_rate=1e-4),
#     loss=total_loss,
#     metrics=metrics
# )

# # Step 4: Continue training
# history = model.fit(
#     X_train,
#     y_train,
#     batch_size=8,
#     epochs=10,  # or more
#     validation_data=(X_test, y_test),
#     verbose=1
# )


In [ ]:
#Load the pretrained model for testing and predictions.
from keras.models import load_model
my_model = load_model('./3D_model_vgg16_2epochs.h5', compile=False)
#If you load a different model do not forget to preprocess accordingly.

In [ ]:
# Predict on the test data
y_pred = my_model.predict(X_test)  # Shape: (batch_size, 64, 64, 64, 8)

# Convert predicted probabilities to class labels
y_pred_argmax = np.argmax(y_pred, axis=-1)  # Shape: (batch_size, 64, 64, 64)

# Convert one-hot encoded y_test to class labels
y_test_argmax = np.argmax(y_test, axis=-1)  # Shape: (batch_size, 64, 64, 64)

# Optional: Confirm shapes
print("Predicted mask shape:", y_pred.shape)
print("y_pred_argmax shape:", y_pred_argmax.shape)
print("y_test_argmax shape:", y_test_argmax.shape)
print("Unique classes " , np.unique(y_pred_argmax))

In [ ]:
from keras.metrics import MeanIoU

# Define number of classes
n_classes = 8 

# Convert to int32 for compatibility
y_test_argmax = y_test_argmax.astype("int32")
y_pred_argmax = y_pred_argmax.astype("int32")

# Initialize MeanIoU metric
IOU_keras = MeanIoU(num_classes=n_classes)

# Compute IoU
IOU_keras.update_state(y_test_argmax.flatten(), y_pred_argmax.flatten())

# Print the result
print("Mean IoU =", IOU_keras.result().numpy())


In [ ]:
import random

test_img_number = random.randint(0, len(X_test) - 1)  

# Select a random test image
test_img = X_test[test_img_number]
ground_truth = y_test[test_img_number]

# Expand dimensions to match model input shape (batch size = 1)
test_img_input = np.expand_dims(test_img, axis=0)  # Shape: (1, 64, 64, 64, 3)

# Apply VGG16-specific preprocessing
test_img_input1 = preprocess_input(test_img_input)  

# Make prediction
test_pred1 = my_model.predict(test_img_input1)

# Convert softmax probabilities to class labels
test_prediction1 = np.argmax(test_pred1, axis=-1)[0]  # Remove batch dim

print("Predicted mask shape:", test_prediction1.shape)  # Expected: (64, 64, 64)

In [ ]:
# Convert one-hot encoded ground truth to class labels
ground_truth_argmax = np.argmax(ground_truth, axis=-1)  # Last axis contains class probabilities

print("Test image shape:", test_img.shape)  # Expected: (64, 64, 64, 3)
print("Ground truth shape:", ground_truth.shape)  # Expected: (64, 64, 64, 8)
print("Ground truth (argmax) shape:", ground_truth_argmax.shape)  # Expected: (64, 64, 64)


In [ ]:
import matplotlib.pyplot as plt

# Define slice index (should be within range)
slice_idx = 14  # Make sure it's within 0 to 63 for a 64x64x64 volume

plt.figure(figsize=(12, 8))

# Display the original test image (grayscale or first channel of RGB)
plt.subplot(131)  # Using 1 row, 3 columns
plt.title('Testing Image')
plt.imshow(test_img[slice_idx, :, :, 0], cmap='gray')  # First channel for visualization

# Display the ground truth segmentation
plt.subplot(132)
plt.title('Ground Truth')
plt.imshow(ground_truth_argmax[slice_idx, :, :], cmap='jet')  # Use 'jet' for segmentation

# Display the predicted segmentation
plt.subplot(133)
plt.title('Model Prediction')
plt.imshow(test_prediction1[slice_idx, :, :], cmap='jet')

plt.tight_layout()
plt.show()


ON FULL VOLUME

In [3]:
import nibabel as nib
import numpy as np
from patchify import patchify

# Load the volume
nii_img = nib.load('./Data_Subset/images/ct_1003_image.nii.gz')
large_image = nii_img.get_fdata()  # shape: (H, W, D)

print("Original volume shape:", large_image.shape)

# Get original depth
depth = large_image.shape[2]
pad_depth = (64 - depth % 64) if depth % 64 != 0 else 0

# Pad along the depth axis (last axis)
padded_image = np.pad(large_image, ((0, 0), (0, 0), (0, pad_depth)), mode='constant', constant_values=0)
print("Padded volume shape:", padded_image.shape)

# Extract non-overlapping patches
patches = patchify(padded_image, (64, 64, 64), step=64)
print("Patches shape:", patches.shape)  # (x, y, z, 64, 64, 64)


Original volume shape: (512, 512, 206)
Padded volume shape: (512, 512, 256)
Patches shape: (8, 8, 4, 64, 64, 64)


In [ ]:
# Predict segmentation mask for each 3D patch
predicted_patches = []

for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        for k in range(patches.shape[2]):
            # Extract a single grayscale patch: shape (64, 64, 64)
            single_patch = patches[i, j, k, :, :, :]

            # Convert to 3-channel input: shape (64, 64, 64, 3)
            single_patch_3ch = np.stack((single_patch,) * 3, axis=-1)

            # Expand dimensions to make it batch-shaped: shape (1, 64, 64, 64, 3)
            single_patch_input = np.expand_dims(single_patch_3ch, axis=0)

            # Preprocess input according to VGG16
            single_patch_input = preprocess_input(single_patch_input)

            # Predict segmentation output: shape (1, 64, 64, 64, 8)
            prediction = my_model.predict(single_patch_input)

            # Convert probabilities to class labels: shape (64, 64, 64)
            prediction_argmax = np.argmax(prediction, axis=4)[0]

            # Append prediction
            predicted_patches.append(prediction_argmax)


In [ ]:
# Convert and reshape prediction list
predicted_patches = np.array(predicted_patches)
print("Predicted patches shape (flat):", predicted_patches.shape)

In [ ]:
predicted_patches_reshaped = np.reshape(predicted_patches,
    (patches.shape[0], patches.shape[1], patches.shape[2], 64, 64, 64)
)
print("Reshaped to patch grid:", predicted_patches_reshaped.shape)

In [ ]:
# Reconstruct segmented volume
reconstructed_image = unpatchify(predicted_patches_reshaped, padded_image.shape)
reconstructed_image = reconstructed_image[:, :, :depth]  # Remove padding
print("Final reconstructed shape:", reconstructed_image.shape)

In [ ]:
# Save basic segmented volume
reconstructed_image = reconstructed_image.astype(np.uint8)
imsave('/content/drive/MyDrive/Colab Notebooks/data/sandstone_3d/all_images/segmented.tif', reconstructed_image)
print("Saved raw segmented image")

In [ ]:
# Create binary masks for each of the 8 classes
num_segments = 8
final = np.zeros((*reconstructed_image.shape, num_segments), dtype=np.int8)

for i in range(num_segments):
    final[..., i] = (reconstructed_image == i).astype(np.int8)

In [ ]:

#Use APEER OMETIFF library to read and write multidimensional images
%pip install apeer-ometiff-library

In [ ]:
from apeer_ometiff_library import io

In [ ]:
# Convert to OMETIFF format (T, Z, C, X, Y)
final = np.expand_dims(final, axis=0)  # Add time axis T=1
final = np.swapaxes(final, 2, 4)       # Move channels to 3rd dimension
print("Shape for OMETIFF (T, Z, C, X, Y):", final.shape)

# Save as OMETIFF
io.write_ometiff("/content/drive/MyDrive/Colab Notebooks/data/sandstone_3d/all_images/segmented_multi_channel.ome.tiff", final)
print("Multi-channel segmented OMETIFF saved successfully!")

In [ ]:
print("Shape of the segmented volume is: T, Z, C, X, Y ", final.shape)
print(final.dtype)